这是一个使用 CodeFuse-Query 分析 Python 项目的教程。在教程中，你将体验到使用命令行工具对代码仓库进行数据化，然后使用 Godel 语言来分析这个仓库。

检查cli是否就绪

In [ ]:
!which sparrow

STEP 0: 克隆要分析的仓库。我们以 python 仓库 [requests](https://github.com/psf/requests.git) 项目为例。

In [ ]:
!git clone https://github.com/psf/requests.git

STEP 1: 代码数据化。使用 `sparrow database create` 命令创建一个db文件，指定待分析的仓库地址（当前目录下的requests子目录），分析的语言（python），以及db文件的存储路径（放置在当前目录下的/db/requests）。执行该命令之后，就会生成一份db文件，该文件存储着代码仓库的结构化数据，之后的分析就是针对这份数据进行。

In [ ]:
!sparrow database create --source-root requests --data-language-type python --output ./db/requests

STEP 2: 使用Godel分析语言分析db文件。在本教程中，可以点击代码左侧的执行按钮，或使用快捷键：`Shift+Enter`，直接运行分析脚本。这里使用 `%db /path/to/db` 魔法命令来设置COREF db路径，内核会读取这个值来进行query查询。

<b>示例</b> 查询 [requests](https://github.com/psf/requests.git) 的文件注释率信息。

第一行通过内核魔法命令指定分析的db路径，后面写查询文件代码注释率 Godel 脚本。

In [ ]:
%db ./db/requests
// script
use coref::python::*

fn default_db() -> PythonDB {
    return PythonDB::load("coref_python_src.db")
}

/**
 * Get cyclomatic complexity of functions
 *
 * @param name   function name
 * @param value  cyclomatic complexity of function
 * @param path   path of file including this function
 * @param sline  function start line
 * @param eline  function end line
 */
fn getCyclomaticComplexity(
    name: string,
    value: int,
    path: string,
    sline: int,
    eline: int) -> bool {
    // get metric function
    for (c in MetricFunction(default_db())) {
        if (path = c.getLocation().getFile().getRelativePath() &&
            name = c.getQualifiedName() &&
            value = c.getCyclomaticComplexity() &&
            sline = c.getLocation().getStartLineNumber() &&
            eline = c.getLocation().getEndLineNumber()) {
            return true
        }
    }
}

fn main() {
    output(getCyclomaticComplexity())
}

保存上一次运行的 query 结果保存到一个JSON文件

In [ ]:
%%save_to ./query.json

STEP 3: 好了，你可以针对分析生成的结果，进行进一步的代码分析了，比如你可以结合pandas库，使用刚刚生成的 query.json 实现最大函数复杂度Top 10的排序查询：

In [ ]:
%%python
import pandas as pd
data = pd.read_json('./query.json')
data.sort_values('value', ascending=False, inplace=True)
top_10 = data.head(10)
print(top_10)

Enjoy！